In [ ]:
!pip install -q transformers accelerate peft datasets bitsandbytes


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model_id = "google/gemma-3-4b-pt"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 131,153,920 || all params: 4,431,233,392 || trainable%: 2.9598


In [ ]:
from datasets import Dataset

with open("lit_texts/vistrel.txt", "r", encoding="utf-8") as f:
    text = f.read()

max_length = 256
stride = 128

tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
samples = []

for i in range(0, len(tokens) - max_length, stride):
    chunk = tokens[i : i + max_length]
    samples.append({"input_ids": chunk})

dataset = Dataset.from_list(samples)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./lora_gemma_vistrel",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    max_steps=30,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000


TrainOutput(global_step=30, training_loss=0.0, metrics={'train_runtime': 34.409, 'train_samples_per_second': 0.872, 'train_steps_per_second': 0.872, 'total_flos': 173042487705600.0, 'train_loss': 0.0, 'epoch': 0.8333333333333334})

In [ ]:
model.save_pretrained("./lora_vistrel_adapter")
tokenizer.save_pretrained("./lora_vistrel_adapter")


('./lora_vistrel_adapter/tokenizer_config.json',
 './lora_vistrel_adapter/special_tokens_map.json',
 './lora_vistrel_adapter/tokenizer.model',
 './lora_vistrel_adapter/added_tokens.json',
 './lora_vistrel_adapter/tokenizer.json')